Önce pandas kütüphanesi ve veritabanını import ediyoruz.

In [ ]:
import pandas as pd

path = "/kaggle/input/competitive-data-science-predict-future-sales/"

urunler = pd.read_csv(path+'/items.csv')
kategoriler = pd.read_csv(path+'/item_categories.csv')
dukkanlar = pd.read_csv(path+'/shops.csv')
satislar = pd.read_csv(path+'/sales_train.csv')
test = pd.read_csv(path+'/test.csv')
orneklem = pd.read_csv(path+'/sample_submission.csv')

print("Veri setleri başarıyla yüklendi.")

Daha sonra ürünlerin satışlarının zaman ve tarih girdilerini ekliyoruz.

In [ ]:
from datetime import datetime
satislar['year'] = pd.to_datetime(satislar['date']).dt.strftime('%Y')
satislar['month'] = satislar.date.apply(lambda x: datetime.strptime(x,'%d.%m.%Y').strftime('%m')) 

satislar.head(2)

Verilerle işlem yapmak için numpy ve sonraki adımlarda kullanacağımız grafikleri görmek için matplot kütüphanesini yüklüyoruz.
Ardından son 3 yıldaki aylara göre ürün satışlarını görüntülüyoruz.

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

%matplotlib inline 

grouped = pd.DataFrame(satislar.groupby(['year','month'])['item_cnt_day'].sum().reset_index())
sns.pointplot(x='month', y='item_cnt_day', hue='year', data=grouped)

Daha sonra tüm ürünlerin ortalama(mean fonksiyonu) olarak son 3 yıldaki fiyat değişimlerini görüntülüyoruz.

In [ ]:
#Fiyat
grouped_price = pd.DataFrame(satislar.groupby(['year','month'])['item_price'].mean().reset_index())
sns.pointplot(x='month', y='item_price', hue='year', data=grouped_price)


Bu grafikte şunu görüyoruz.
*     Yılın son iki ayı genelde daha fazla satış oluyor.
*     2015 yılında da daha fazla satış bekliyoruz.


Sırada önceki 3 yılı birleştirip zamana göre satış trendini belirlemek var.

In [ ]:
ts=satislar.groupby(["date_block_num"])["item_cnt_day"].sum()
ts.astype('float')
plt.figure(figsize=(16,8))
plt.title('Bütün zaman periyodu boyunca olan bütün satışlar')
plt.xlabel('Zaman')
plt.ylabel('Satışlar')
plt.plot(ts);

Ürün başına gerçekleşen ortalama satışı, bunun ortalamasını, standart sapma değerini, minimum ve maksimum değerlerini buluyoruz.

In [ ]:
satislar.item_cnt_day.hist(bins=100)
satislar.item_cnt_day.describe()

Çıktılar bize eksi değerde satışlar olduğunu gösteriyor. Bunları veri setinden çıkarmamız lazım.

In [ ]:
print('Fiyatı sıfırdan düşük olan ürünler silinmeden önceki veri seti büyüklüğü:', satislar.shape)
satislar = satislar.query('item_price > 0')
print('Fiyatı sıfırdan düşük olan ürünler silindikten sonraki veri seti büyüklüğü:', satislar.shape)

Sadece 1 adet negatif değerli ürün olduğunu görüyoruz.
Ardından tekrar eden ürün ve dükkanları kaldırmak için unique(özel) fonksiyonunu kullanarak yeni bir filtreleme gerçekleştiriyoruz.

In [ ]:
print('Filtrelemeden önceki veri seti büyüklüğü:', satislar.shape)
# Test setinde sadece dükkanlar bulunuyor.
satislar = satislar[satislar['shop_id'].isin(test['shop_id'].unique())]
# Test setinde sadece ürünler bulunuyor.
satislar = satislar[satislar['item_id'].isin(test['item_id'].unique())]
print('Filtrelemeden sonraki veri seti büyüklüğü:', satislar.shape)

Elimizdeki toplam ürün sayısı 1.224.439
Ardından filtremizi biraz daha geliştirip günlük ürün satışı 0'dan büyük ve 125'ten küçük olan fiyatıda 75000'i geçmeyen ürünleri buluyoruz.

In [ ]:
print('Aykırı değerler silinmeden önceki veri seti büyüklüğü:', satislar.shape)
satislar = satislar.query('item_cnt_day >= 0 and item_cnt_day <= 125 and item_price < 75000')
print('Aykırı değerler silindikten sonraki veri seti büyüklüğü:', satislar.shape)

Satışları aylık olarak topluyoruz.

In [ ]:
aylik_satislar=satislar.groupby(["date_block_num","shop_id","item_id"])["date_block_num","date","item_price","item_cnt_day"].agg({"date_block_num":'mean',"date":["min",'max'],"item_price":"mean","item_cnt_day":"sum"})
aylik_satislar.head(5)

Çıktılar çok uzun olduğu için sadece 5 ürünü örnek olarak listeledik.

In [ ]:
duzgun_satis_verisi = aylik_satislar.item_cnt_day.apply(list).reset_index()
#Sadece düzgün veriyle işlem yapacağız.
duzgun_satis_verisi = pd.merge(test,duzgun_satis_verisi,on = ['item_id','shop_id'],how = 'left')
#NA değerleri 0 ile dolduruyoruz.
duzgun_satis_verisi.fillna(0,inplace = True)
duzgun_satis_verisi.drop(['shop_id','item_id'],inplace = True, axis = 1)
duzgun_satis_verisi.head(20)

Gereksiz verileri kaldırıp sadece zaman damgası ve ürün idlere göre satış sayılarını bırakıyoruz.
Ardından kerasta kullanmak için pivot tablosu oluşturucağız.

In [ ]:
# Satırlar = Her dükkan ve ürün kodunun birleşimi
# Sütunlar = Zaman sekansı olarak kullanılacak.
pivot_tablosu = duzgun_satis_verisi.pivot_table(index='ID', columns='date_block_num',fill_value = 0,aggfunc='sum' )
pivot_tablosu.head(20)

Artık elimizde tam istediğimiz gibi bir matris tablosu var.

In [ ]:
# Son sütun hariç hepsini kullanacağız.
X_train = np.expand_dims(pivot_tablosu.values[:,:-1],axis = 2)
# Çünkü son sütun bizim tahminlememiz olacak.
y_train = pivot_tablosu.values[:,-1:]

# Test içinse ilk sütun hariç diğer bütün sütunları kullanacağız.
X_test = np.expand_dims(pivot_tablosu.values[:,1:],axis = 2)

# Şekle bir göz atalım.
print(X_train.shape,y_train.shape,X_test.shape)

Veriyi işlemek için önce bir kaç farklı deneme yaptık fakat sıralı modelde karar kıldık.

Sequential(sıralı)model, her katmanın tam olarak bir giriş tensörü ve bir çıkış tensörüne sahip olduğu düz bir katman yığını için uygundur. 
Bizimde sadece fiyat ve ürün sayısı değişkenlerimiz olduğu için bunu seçtik.
LSTM, dropout ve dense değerlerini belirleyip, matrisi şekillendirince elimizde kaç farklı parametre olduğunu görüyoruz.

LTSM=Kısa-Uzun süreli hafıza     
DENSE=Katman yoğunluğu
DROPOUT=Seyreltme katmanı

LTSM = Yapay sinir ağları eğitiminde giriş verileriyle çıkış değerleri hesaplanır. Hesaplanmış çıkış değerleri ile gerçek değerler karşılaştırılır ve farklılık bize bir  meyil  - gradient - Bu meyile göre geriye doğru ağ ağırlık değerlerinde güncelleme yapılır. Ağımız çok karmaşıksa geriye doğru meyil değerleri sıfırlanır. Meyil olmazsada güncelleme olmaz ve eğitilmeside durmuş olur. RNN yapısında da zaman içinde geriye doğru bağlılık var. Ağımızı eğitim açısında çok karmaşık hale getiriryor.Bu problemin çözümü için LSTM - Long short-term memory - denen bir yapı kurulmuş.

DENSE = Eklediğimiz katman Keras’ın Dense layer tipinde. Bu “fully connected layer”(tamamen bağlı katman) olarak da biliniyor.
Yani bu katmanın bütün girişleri ve bütün çıkışları birbirine bağlı ve her birinin ayrı bir ağırlığı(weight) var.

DROPOUT = Aşırı öğrenmeyi(overfitting) engellemek için bazı nöronları unutmak kullanılanılır.

In [ ]:
from keras.models import Sequential
from keras.layers import LSTM,Dense,Dropout
from keras.models import load_model, Model

# Satış modelini tanımlıyoruz.
satis_modeli = Sequential()
satis_modeli.add(LSTM(units = 64,input_shape = (33,1)))
satis_modeli.add(Dropout(0.5))
satis_modeli.add(Dense(1))

satis_modeli.compile(loss = 'mse',optimizer = 'adam', metrics = ['mean_squared_error'])
satis_modeli.summary()

Daha sonra batch size belirleyip 20 adımda tahminlemeyi gerçekleştiriyoruz.

Batch size = Örnek verilerin kaçarlı gruplar halinde nöron içine girdiği belirlemek için kullanılır.

Biz verisetimiz çok büyük olduğu için 4096 olarak ayarladık.

In [ ]:
satis_modeli.fit(X_train,y_train,batch_size = 4096,epochs = 20)

Son olarak işlenen verileri csv dosyası şeklinde çıktı alarak her ürünün adet olarak aylık ortalama kaç tane satılacağını buluyoruz.

In [ ]:
orneklem_ciktisi = satis_modeli.predict(X_test)
orneklem = pd.DataFrame({'ID':test['ID'],'tahmini_aylik_satis':orneklem_ciktisi.ravel()})

orneklem.to_csv('tahminleme.csv',index = False)
orneklem.head(100)

İlk 100 ürünü örnek olarak yazdırdık ve istersek tahminleme.csv dosyamızdan bütün ürün satış verilerine ulaşabiliriz.